In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [3]:
filename = "C:/Projects/Text Generation/macbeth.txt"
text = (open(filename).read()).lower()

In [5]:
# mapping characters with integers
unique_chars = sorted(list(set(text)))

In [7]:
#creating character to integer mappings and int to char mapping

char_to_int = {}
int_to_char = {}

for i, c in enumerate (unique_chars):
    char_to_int.update({c: i})
    int_to_char.update({i: c})

In [10]:
# preparing input and output dataset
X = []
Y = []

for i in range(0, len(text) - 50, 1):
    sequence = text[i:i + 50]
    label =text[i + 50]
    X.append([char_to_int[char] for char in sequence])
    Y.append(char_to_int[label])

In [29]:
print(len(X))
print(len(Y))

100270
100270


In [13]:
# reshaping, normalizing and one hot encoding
X_modified = np.reshape(X, (len(X), 50, 1))
X_modified = X_modified / float(len(unique_chars))
Y_modified = np_utils.to_categorical(Y)

In [18]:
print(X_modified.shape)
print(Y_modified.shape)

(100270, 50, 1)
(100270, 43)


In [30]:
# defining the LSTM model
model = Sequential()
model.add(LSTM(300, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0810 01:37:28.447800  3628 deprecation_wrapper.py:119] From C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 01:37:28.707219  3628 deprecation_wrapper.py:119] From C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 01:37:28.722310  3628 deprecation_wrapper.py:119] From C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 01:37:29.139528  3628 deprecation_wrapper.py:119] From C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_

In [47]:
# fitting the model
model.fit(X_modified, Y_modified, epochs=1, batch_size=30)

Epoch 1/1
100270/100270 [==============================] - 1287s 13ms/step - loss: 2.3850


In [48]:
# picking a random seed
start_index = numpy.random.randint(0, len(X)-1)
new_string = X[start_index]

In [50]:
# generating characters
for i in range(50):
    x = np.reshape(new_string, (1, len(new_string), 1))
    x = x / float(len(unique_chars))

    #predicting
    pred_index = np.argmax(model.predict(x, verbose=0))
    char_out = int_to_char[pred_index]
    seq_in = [int_to_char[value] for value in new_string]
    new_string.append(pred_index)
    new_string = new_string[1:len(new_string)]